In [1]:
import pandas as pd
import seaborn as sns
import mlflow

In [2]:
import numpy as np

In [3]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiments')

<Experiment: artifact_location='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1', creation_time=1743230295117, experiment_id='1', last_update_time=1743230295117, lifecycle_stage='active', name='nyc-taxi-experiments', tags={}>

In [5]:
df=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('yellow_tripdata_2024-01.parquet')
df_val = read_dataframe('yellow_tripdata_2024-02.parquet')

In [7]:
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,19.800000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,6.600000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,17.916667
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,8.300000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,6.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,8.616667
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,14.750000
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,19.000000
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,17.850000


In [8]:
len(df_train), len(df_val)

(2898906, 2938060)

In [9]:

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

30.353066020774595

In [13]:
import os
os.makedirs("models", exist_ok=True)

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("Machine Learning Engineer", "Fahlevi")

    mlflow.log_param("train-data-path", "yellow_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "yellow_tripdata_2024-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=50,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 50, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

c:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [14:20:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.84311                          
[1]	validation-rmse:5.75467                          
[2]	validation-rmse:5.43692                          
[3]	validation-rmse:5.32940                          
[4]	validation-rmse:5.28716                          
[5]	validation-rmse:5.26351                          
[6]	validation-rmse:5.25158                          
[7]	validation-rmse:5.24127                          
[8]	validation-rmse:5.23609                          
[9]	validation-rmse:5.22991                          
[10]	validation-rmse:5.22577                         
[11]	validation-rmse:5.22186                         
[12]	validation-rmse:5.21769                         
[13]	validation-rmse:5.21411                         
[14]	validation-rmse:5.21062                         
[15]	validation-rmse:5.20655                         
[16]	validation-rmse:5.20358                         
[17]	validation-rmse:5.20014                         
[18]	validation-rmse:5.19661

In [21]:
params={
    'learning_rate':0.5315026836051042,
'max_depth': 33,
'min_child_weight':2.3065027888594636,
'objective': 'reg:linear',
'reg_alpha':0.007106204421482791,
'reg_lambda':0.005102984304705061,
'seed':42
}

In [22]:
mlflow.xgboost.autolog() 
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=50,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)
y_pred = booster.predict(valid)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

2025/03/29 14:37:39 WARNING mlflow.utils.autologging_utils: MLflow xgboost autologging is known to be compatible with 1.4.2 <= xgboost <= 2.1.4, but the installed version is 3.0.0. If you encounter errors during autologging, try upgrading / downgrading xgboost to a compatible version, or try upgrading MLflow.
2025/03/29 14:37:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ee52ad8079dc48d19bb11c97db906289', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/03/29 14:37:39 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 259. GiB for an array with shape (2898906, 24016) and data type float32
c:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [14:37:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now de

[0]	validation-rmse:6.84311
[1]	validation-rmse:5.75467
[2]	validation-rmse:5.43692
[3]	validation-rmse:5.32940
[4]	validation-rmse:5.28716
[5]	validation-rmse:5.26351
[6]	validation-rmse:5.25158
[7]	validation-rmse:5.24127
[8]	validation-rmse:5.23609
[9]	validation-rmse:5.22991
[10]	validation-rmse:5.22577
[11]	validation-rmse:5.22186
[12]	validation-rmse:5.21769
[13]	validation-rmse:5.21411
[14]	validation-rmse:5.21062
[15]	validation-rmse:5.20655
[16]	validation-rmse:5.20358
[17]	validation-rmse:5.20014
[18]	validation-rmse:5.19661
[19]	validation-rmse:5.19296
[20]	validation-rmse:5.18960
[21]	validation-rmse:5.18679
[22]	validation-rmse:5.18428
[23]	validation-rmse:5.17879
[24]	validation-rmse:5.17553
[25]	validation-rmse:5.17308
[26]	validation-rmse:5.17070
[27]	validation-rmse:5.16843
[28]	validation-rmse:5.16620
[29]	validation-rmse:5.16333
[30]	validation-rmse:5.16114
[31]	validation-rmse:5.15923
[32]	validation-rmse:5.15622
[33]	validation-rmse:5.15427
[34]	validation-rmse:5.1

2025/03/29 14:40:18 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/03/29 14:40:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [14:40:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/03/29 14:40:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [20]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    best_params={
    'learning_rate':0.5315026836051042,
'max_depth': 33,
'min_child_weight':2.3065027888594636,
'objective': 'reg:linear',
'reg_alpha':0.007106204421482791,
'reg_lambda':0.005102984304705061,
'seed':42
}
    mlflow.log_params(best_params) 

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=50,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse",rmse)
    
    with open('models/preprocessor.b',"wb") as f_out:
        pickle.dump(dv,f_out)
    
    mlflow.log_artifact('models/preprocessor.b',artifact_path='preprocessor')
    
    mlflow.xgboost.log_model(booster,artifact_path='models_mlflow')

c:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\.venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [15:51:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:6.84311
[1]	validation-rmse:5.75467
[2]	validation-rmse:5.43692
[3]	validation-rmse:5.32940
[4]	validation-rmse:5.28716
[5]	validation-rmse:5.26351
[6]	validation-rmse:5.25158
[7]	validation-rmse:5.24127
[8]	validation-rmse:5.23609
[9]	validation-rmse:5.22991
[10]	validation-rmse:5.22577
[11]	validation-rmse:5.22186
[12]	validation-rmse:5.21769
[13]	validation-rmse:5.21411
[14]	validation-rmse:5.21062
[15]	validation-rmse:5.20655
[16]	validation-rmse:5.20358
[17]	validation-rmse:5.20014
[18]	validation-rmse:5.19661
[19]	validation-rmse:5.19296
[20]	validation-rmse:5.18960
[21]	validation-rmse:5.18679
[22]	validation-rmse:5.18428
[23]	validation-rmse:5.17879
[24]	validation-rmse:5.17553
[25]	validation-rmse:5.17308
[26]	validation-rmse:5.17070
[27]	validation-rmse:5.16843
[28]	validation-rmse:5.16620
[29]	validation-rmse:5.16333
[30]	validation-rmse:5.16114
[31]	validation-rmse:5.15923
[32]	validation-rmse:5.15622
[33]	validation-rmse:5.15427
[34]	validation-rmse:5.1

c:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [15:56:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/03/29 15:57:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [22]:
import mlflow
logged_model = 'runs:/70ac6f7b08ef42b3b4e7115810836c49/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [23]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 70ac6f7b08ef42b3b4e7115810836c49

In [24]:
xgboost_model=mlflow.xgboost.load_model(logged_model)

In [25]:
xgboost_model

In [26]:
    y_pred = xgboost_model.predict(valid)
    y_pred[:10]

array([24.276075 , 21.112692 , 44.302418 ,  9.859947 , 16.920456 ,
        6.8124347, 17.273108 , 14.940294 , 10.124242 , 10.354489 ],
      dtype=float32)